# 1- input and data wrangling

In [ ]:
# Import imported libraries
import pandas as pd
from google.colab import drive
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta

# connect to Google drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# # Reading the dataset
# df_all = df = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/Scraped Motley Fool Earnings Call Transcripts/motley-fool-data.pkl')

In [ ]:
# # save the dataset as csv
# df_all.to_csv("/content/drive/MyDrive/Colab Notebooks/Scraped Motley Fool Earnings Call Transcripts/motley-fool-data.csv", index=False)

_______________________________________________________________________________________________

In [ ]:
# Read the data
df_all = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Scraped Motley Fool Earnings Call Transcripts/motley-fool-data.csv')

In [ ]:
# look at the DataFrame
df_all.head()

,date,exchange,q,ticker,transcript
0,"Aug 27, 2020, 9:00 p.m. ET",NASDAQ: BILI,2020-Q2,BILI,"Prepared Remarks:\nOperator\nGood day, and wel..."
1,"Jul 30, 2020, 4:30 p.m. ET",NYSE: GFF,2020-Q3,GFF,Prepared Remarks:\nOperator\nThank you for sta...
2,"Oct 23, 2019, 5:00 p.m. ET",NASDAQ: LRCX,2020-Q1,LRCX,Prepared Remarks:\nOperator\nGood day and welc...
3,"Nov 6, 2019, 12:00 p.m. ET",NASDAQ: BBSI,2019-Q3,BBSI,"Prepared Remarks:\nOperator\nGood day, everyon..."
4,"Aug 7, 2019, 8:30 a.m. ET",NASDAQ: CSTE,2019-Q2,CSTE,Prepared Remarks:\nOperator\nGreetings and wel...


In [ ]:
# look at transcript
df_all.transcript[0]

"Prepared Remarks:\nOperator\nGood day, and welcome to the Bilibili 2020 Second Quarter Earnings Conference Call. Today's conference is being recorded.\nAt this time, I would like to turn the conference over to Juliet Yang, Senior Director of Investor Relations. Please go ahead.\nJuliet Yang -- Senior Director of Investor Relations\nThank you, operator.\nPlease note the discussion today will contain forward-looking statements relating to the Company's future performance, and are intended to qualify for the Safe Harbor from liability, as established by the US Private Securities Litigation Reform Act. Such statements are not guarantees of future performance and are subject to certain risks and uncertainties, assumptions and other factors. Some of these risks are beyond the Company's control and could cause actual results to differ materially from those mentioned in today's press release and this discussion. A general discussion of the risk factors that could affect Bilibili's business an

In [ ]:
# change date column to datetime
df_all['date2'] = pd.to_datetime(df_all['date'],errors='coerce')

# not converted
df_all[df_all['date2'].isna()]['date']

/usr/local/lib/python3.10/dist-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname ET identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


28       ['Brunswick (BC 0.66%) Q4 2018 ', 'Jan. 31, 20...
125      ['Blucora (BCOR 1.57%) Q4 2018 ', 'Feb. 14, 20...
154      ['Hoegh LNG Partners LP\xa0 (HMLP)Q4\xa02018 '...
161      ['AECOM Technology (ACM 0.87%) Q1 2019 ', 'Feb...
235      ['GATX Corp\xa0 (GATX 0.43%)Q4\xa02018 ', 'Jan...
                               ...                        
10936    ['Mercury Computer Systems (MRCY -0.77%) Q2 20...
10969    ['Marinemax Inc\xa0 (HZO 0.27%)Q1\xa02019 ', '...
10989    ['Deere & Company (DE 7.53%) Q4 2017 ', 'Nov. ...
11001    ['Reliance Steel And Aluminum Co\xa0 (RS 1.97%...
11043    ['Ball Corp (BALL 0.26%) Q4 2018 ', 'Jan. 31, ...
Name: date, Length: 380, dtype: object

In [ ]:
# Remove unwanted characters from date column
df_all.loc[df_all.isnull().any(axis=1), 'date'] = df_all.loc[df_all.isnull().any(axis=1),'date'].str.replace(r"'.*,", "").str.slice(2, -2)

df_all[df_all['date2'].isna()]['date']

<ipython-input-8-ef3cee549ce9>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_all.loc[df_all.isnull().any(axis=1), 'date'] = df_all.loc[df_all.isnull().any(axis=1),'date'].str.replace(r"'.*,", "").str.slice(2, -2)


28       2019 11:00 a.m. ET
125       2019 8:30 a.m. ET
154            8:30 a.m. ET
161      2019 12:00 p.m. ET
235           11:00 a.m. ET
                ...        
10936     2018 5:00 p.m. ET
10969         10:00 a.m. ET
10989    2017 10:00 a.m. ET
11001         11:00 a.m. ET
11043    2019 11:00 a.m. ET
Name: date, Length: 380, dtype: object

In [ ]:
# convert the rest and see if there is any other nulls
df_all['date2'] = pd.to_datetime(df_all['date'],errors='coerce')
df_all[df_all['date2'].isna()]['date']

/usr/local/lib/python3.10/dist-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname ET identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


416                          NaN
2965        Domino 10:00 a.m. ET
6946            xa010:00 a.m. ET
6960      McDonald 10:00 a.m. ET
7406             O 11:00 a.m. ET
7553      McDonald 11:00 a.m. ET
8310             xa09:00 a.m. ET
9240             O 11:00 a.m. ET
9763             xa09:00 a.m. ET
10646     McDonald 11:00 a.m. ET
10787    Alexander 10:00 a.m. ET
Name: date, dtype: object

In [ ]:
# remove the other nulls
df = df_all.dropna()
df[df['date2'].isna()]

,date,exchange,q,ticker,transcript,date2


In [ ]:
# Replace the date column we create with the old one
df = df.drop('date', axis=1)
df = df.rename(columns={'date2': 'date'})
df.head()

,exchange,q,ticker,transcript,date
0,NASDAQ: BILI,2020-Q2,BILI,"Prepared Remarks:\nOperator\nGood day, and wel...",2020-08-27 21:00:00
1,NYSE: GFF,2020-Q3,GFF,Prepared Remarks:\nOperator\nThank you for sta...,2020-07-30 16:30:00
2,NASDAQ: LRCX,2020-Q1,LRCX,Prepared Remarks:\nOperator\nGood day and welc...,2019-10-23 17:00:00
3,NASDAQ: BBSI,2019-Q3,BBSI,"Prepared Remarks:\nOperator\nGood day, everyon...",2019-11-06 12:00:00
4,NASDAQ: CSTE,2019-Q2,CSTE,Prepared Remarks:\nOperator\nGreetings and wel...,2019-08-07 08:30:00


In [ ]:
# Create important functions to get the stock price
def get_stock_price(row):
  try:
    # Construct the full ticker
    full_ticker = f"{row['date']}-{row['ticker']}"
    symbol = row['ticker']
    date = row['date']
    data = yf.download(tickers = symbol, start=date, end=date + timedelta(days=3))
    # print(data['Close'])
    return data['Close'][0]
  except:
    return np.NaN

def get_stock_price_1Q_Later(row):
  try:
    # Construct the full ticker
    full_ticker = f"{row['date']}-{row['ticker']}"
    symbol = row['ticker']
    date = row['date']

    data = yf.download(tickers = symbol, start = date + timedelta(days=90), end = date + timedelta(days=93))
    # print(data['Close'])
    return data['Close'][0]
  except:
    return np.NaN

df['stock_price'] = df.apply(get_stock_price, axis=1)
df['stock_price_1Q_Later'] = df.apply(get_stock_price_1Q_Later, axis=1)


In [ ]:
# Count Nulls
len(df[df['stock_price'].isna()])

1626

In [ ]:
# Remove Nulls
df = df.dropna()

In [ ]:
# Create price_change column for changes more than the threshold
# Define the threshold as the largest neglected change (can be changed according to the business case)
threshold = 5
df["price_change"] = np.where(df["stock_price_1Q_Later"] - df["stock_price"]> threshold, 0,
                              np.where(df["stock_price"] - df["stock_price_1Q_Later"] > threshold, 1, 2))


    Price Changes:
    "Increase": 0,
    "Decrease": 1,
    "neutral": 2

In [ ]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/Scraped Motley Fool Earnings Call Transcripts/df_final.csv', index=False)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Scraped Motley Fool Earnings Call Transcripts/df_final.csv')

In [ ]:
df.head(50)

,exchange,q,ticker,transcript,date,stock_price,stock_price_1Q_Later,price_change
0,NASDAQ: BILI,2020-Q2,BILI,"Prepared Remarks:\nOperator\nGood day, and wel...",2020-08-27 21:00:00,47.389999,64.860001,0
1,NYSE: GFF,2020-Q3,GFF,Prepared Remarks:\nOperator\nThank you for sta...,2020-07-30 16:30:00,20.639999,21.900000,2
2,NASDAQ: LRCX,2020-Q1,LRCX,Prepared Remarks:\nOperator\nGood day and welc...,2019-10-23 17:00:00,233.179993,308.679993,0
3,NASDAQ: BBSI,2019-Q3,BBSI,"Prepared Remarks:\nOperator\nGood day, everyon...",2019-11-06 12:00:00,93.750000,84.570000,1
4,NASDAQ: CSTE,2019-Q2,CSTE,Prepared Remarks:\nOperator\nGreetings and wel...,2019-08-07 08:30:00,15.440000,17.020000,2
5,NYSE: GDOT,2020-Q3,GDOT,"Prepared Remarks:\nOperator\nGood afternoon, a...",2020-11-04 17:00:00,55.000000,55.250000,2
6,NYSE: KAR,2020-Q2,KAR,Prepared Remarks:\nOperator\nLadies and gentle...,2020-08-05 08:30:00,16.650000,16.309999,2
7,NYSE: EPAC,2021-Q2,EPAC,Prepared Remarks:\nOperator\nLadies and gentle...,2021-03-24 11:00:00,23.549999,25.280001,2
8,NYSE: ABM,2021-Q2,ABM,Prepared Remarks:\nOperator\nGreetings and wel...,2021-06-09 08:30:00,48.540001,47.220001,2
9,NYSE: SGHC,2022-Q2,SGHC,Prepared Remarks:\nOperator\nGood morning and ...,2022-08-11 08:30:00,4.970000,3.450000,2


# 2- BERT Classifier (stock price change)

In [ ]:
!pip install transformers
!pip install datasets
!pip install sparknlp
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 64.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.3/531.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=b38a55a3516c00cf45c51a566980b2e80e8b7b6c5f283c552736127f2da5

In [ ]:
# Import libraries
import torch
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
from transformers import BertForSequenceClassification, BertTokenizer
import transformers
import torch


In [ ]:
# Check if CUDA is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla T4


In [ ]:
# Load pre-trained BERT model and tokenizer
model = transformers.BertModel.from_pretrained('bert-base-uncased')
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
# Define data processing functions
def tokenize(text):
  # Tokenize the text using BERT tokenizer
  tokens = tokenizer.tokenize(text)
  # Add special tokens [CLS] and [SEP]
  tokens = ['[CLS]'] + tokens + ['[SEP]']
  # Convert tokens to ids
  ids = tokenizer.convert_tokens_to_ids(tokens)
  return ids

def pad_and_truncate(ids, max_len):
  # Pad or truncate the ids to a fixed length
  if len(ids) < max_len:
    # Pad with zeros
    ids = ids + [0] * (max_len - len(ids))
  else:
    # Truncate from the end
    ids = ids[:max_len]
  return ids

def create_attention_mask(ids):
  # Create an attention mask that ignores padded tokens
  mask = [int(i > 0) for i in ids]
  return mask

In [ ]:
# Load stock price data and text data
stock_df = df.price_change
text_df = df.transcript

In [ ]:
# Split data into train, val, and test sets
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

train_size = int(len(stock_df) * train_ratio)
val_size = int(len(stock_df) * val_ratio)
test_size = int(len(stock_df) * test_ratio)

train_stock_df = stock_df[:train_size]
val_stock_df = stock_df[train_size:train_size+val_size]
test_stock_df = stock_df[train_size+val_size:]

train_text_df = text_df[:train_size]
val_text_df = text_df[train_size:train_size+val_size]
test_text_df = text_df[train_size+val_size:]

In [ ]:
# Define hyperparameters
# Maximum length of input sequence
max_len = 512
# Batch size for training
batch_size = 16
# Number of epochs for training
num_epochs = 6
# Learning rate for optimizer
learning_rate = 2e-5

In [ ]:
# Process text data using BERT tokenizer
train_ids = train_text_df.apply(tokenize).apply(pad_and_truncate, args=(max_len,))
val_ids = val_text_df.apply(tokenize).apply(pad_and_truncate, args=(max_len,))
test_ids = test_text_df.apply(tokenize).apply(pad_and_truncate, args=(max_len,))
# ids = text_df.apply(tokenize).apply(pad_and_truncate, args=(max_len,))


train_masks = train_ids.apply(create_attention_mask)
val_masks = val_ids.apply(create_attention_mask)
test_masks = test_ids.apply(create_attention_mask)
# masks = ids.apply(create_attention_mask)


In [ ]:
# Convert data to tensors
train_ids = torch.tensor(train_ids.tolist())
val_ids = torch.tensor(val_ids.tolist())
test_ids = torch.tensor(test_ids.tolist())
# ids = torch.tensor(ids.tolist())

train_masks = torch.tensor(train_masks.tolist())
val_masks = torch.tensor(val_masks.tolist())
test_masks = torch.tensor(test_masks.tolist())
# masks = torch.tensor(masks.tolist())

train_labels = torch.tensor(train_stock_df.tolist())
val_labels = torch.tensor(val_stock_df.tolist())
test_labels = torch.tensor(test_stock_df.tolist())
# labels = torch.tensor(stock_df.tolist())

In [ ]:
# Create data loaders
train_data = torch.utils.data.TensorDataset(train_ids, train_masks, train_labels)
val_data = torch.utils.data.TensorDataset(val_ids, val_masks, val_labels)
test_data = torch.utils.data.TensorDataset(test_ids, test_masks, test_labels)
# data = torch.utils.data.TensorDataset(ids, masks, labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)
# loader = torch.utils.data.DataLoader(data, batch_size=len(text_df), shuffle=False)

In [ ]:
# Define model architecture
class BertStockPredictor(torch.nn.Module):
  def __init__(self, num_classes):
    super(BertStockPredictor, self).__init__()
    # Use pre-trained BERT model
    self.bert = model
    # Add a softmax layer on top of BERT output
    self.softmax = torch.nn.Linear(model.config.hidden_size, num_classes)
    self.activation = torch.nn.Softmax(dim=1)

  def forward(self, input_ids, attention_mask):
    # Get the last hidden state from BERT
    output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
    last_hidden_state = output.last_hidden_state
    # Get the [CLS] token representation
    cls_rep = last_hidden_state[:, 0, :]
    # Pass it through the softmax layer
    logits = self.softmax(cls_rep)
    pred = self.activation(logits)
    return pred

# Instantiate the model with the number of classes you have
num_classes = 3
model = BertStockPredictor(num_classes)
# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


BertStockPredictor(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [ ]:
# Define loss function and optimizer
loss_fn = torch.nn.CrossEntropyLoss() # Cross entropy loss
optimizer = transformers.AdamW(model.parameters(), lr=learning_rate) # AdamW optimizer

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
torch.cuda.empty_cache()

In [ ]:
# Train the model
best_loss = 10

best_val_loss = float('inf') # Initialize the best validation loss
for epoch in range(num_epochs):
  # Set the model to training mode
  model.train()
  # Initialize the training loss and accuracy
  train_loss = 0.0
  train_acc = 0.0
  # Loop over the training batches
  for batch in train_loader:
    # Get the input ids, masks, and labels from the batch
    input_ids, attention_mask, labels = batch
    # Move them to the device
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    labels = labels.to(device)
    # Clear the gradients
    optimizer.zero_grad()
    # Forward pass
    preds = model(input_ids, attention_mask).squeeze()
    # Compute the loss
    loss = loss_fn(preds, labels)
    # Backward pass and update the parameters
    loss.backward()
    optimizer.step()
    # Accumulate the training loss and accuracy
    train_loss += loss.item()
  # Compute the average training loss and accuracy
  train_loss /= len(train_loader)

  # Set the model to evaluation mode
  model.eval()
  # Initialize the validation loss and accuracy
  val_loss = 0.0
  val_acc = 0.0

  # Loop over the validation batches
  for batch in val_loader:
    # Get the input ids, masks, and labels from the batch
    input_ids, attention_mask, labels = batch
    # Move them to the device
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    labels = labels.to(device)
    # Forward pass with no gradient calculation
    with torch.no_grad():
      preds = model(input_ids, attention_mask).squeeze()
      # Compute the loss
      loss = loss_fn(preds, labels)
      # Accumulate the validation loss
      val_loss += loss.item()
  # Compute the average validation loss
  val_loss /= len(val_loader)

  # Print the epoch summary
  print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
  # save the best model
  if val_loss < best_loss:
    best_loss = val_loss
    torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/Scraped Motley Fool Earnings Call Transcripts/best_model_1.pth')


Epoch 1, Train Loss: 0.8283, Val Loss: 0.9222
Epoch 2, Train Loss: 0.8127, Val Loss: 0.9246
Epoch 3, Train Loss: 0.7961, Val Loss: 0.9128
Epoch 4, Train Loss: 0.7850, Val Loss: 0.9248
Epoch 5, Train Loss: 0.7689, Val Loss: 0.9237
Epoch 6, Train Loss: 0.7610, Val Loss: 0.9196


In [ ]:
# load the best model state dictionary
model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/Scraped Motley Fool Earnings Call Transcripts/best_model_1.pth'))

# # for CPU:
# model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/Scraped Motley Fool Earnings Call Transcripts/best_model.pth', map_location=torch.device('cpu')))


<All keys matched successfully>

In [ ]:
# Set the model to evaluation mode
model.eval()
# Initialize the test loss and accuracy
test_loss = 0
test_acc = 0
# Loop over the Test batches
for batch in test_loader:
    # Get the input ids, masks, and labels from the batch
    input_ids, attention_mask, labels = batch
    # Move them to the device
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    labels = labels.to(device)
    # Forward pass with no gradient calculation
    with torch.no_grad():
      preds = model(input_ids, attention_mask).squeeze()
      # Compute the loss
      loss = loss_fn(preds, labels)
      # Accumulate the Test loss and accuracy
      test_loss += loss.item()

# Compute the average Test loss and accuracy
test_loss /= len(test_loader)
print(f'test Loss: {test_loss:.4f}')

test Loss: 0.9178


In [ ]:
#  # Save Loaders
#  for batch in loader:
#     # Get the input ids, masks, and labels from the batch
#     input_ids, attention_mask, labels = batch
#     # Save `input_ids` to a file
#     with open('/content/drive/MyDrive/Colab Notebooks/Scraped Motley Fool Earnings Call Transcripts/input_ids.txt', 'w') as f:
#         for item in input_ids.tolist():
#             f.write("%s\n" % item)

#     # Save `attention_mask` to a file
#     with open('/content/drive/MyDrive/Colab Notebooks/Scraped Motley Fool Earnings Call Transcripts/attention_mask.txt', 'w') as f:
#         for item in attention_mask.tolist():
#             f.write("%s\n" % item)

#     # Save `labels` to a file
#     with open('/content/drive/MyDrive/Colab Notebooks/Scraped Motley Fool Earnings Call Transcripts/labels.txt', 'w') as f:
#         for item in labels.tolist():
#             f.write("%s\n" % item)


In [ ]:
# Load `input_ids` from file
df1 = pd.read_table("/content/drive/MyDrive/Colab Notebooks/Scraped Motley Fool Earnings Call Transcripts/input_ids.txt", header=None)
input1= list(df1.iloc[:,0])

input_ids = []
for sublist in eval(str(input1)):
  list_a = list(map(int, eval(sublist)))
  input_ids.append(list_a)



# Load `attention_mask` from file
df2 = pd.read_table("/content/drive/MyDrive/Colab Notebooks/Scraped Motley Fool Earnings Call Transcripts/attention_mask.txt", header=None)
input2= list(df2.iloc[:,0])

attention_mask = []
for sublist in eval(str(input2)):
  list_a = list(map(int, eval(sublist)))
  attention_mask.append(list_a)



# Load `labels` from file
with open('/content/drive/MyDrive/Colab Notebooks/Scraped Motley Fool Earnings Call Transcripts/labels.txt', 'r') as f:
    labels = [int(line.rstrip('\n')) for line in f]



In [ ]:
# predect uusing small patches (because of small memory and disk space)
def split_ten(input , patch_size):
  input_patches = []
  for i in range(int(len(input)/patch_size)+1):
    t = torch.tensor(input[patch_size * i : patch_size * (i+1)])
    input_patches.append(t)
  return input_patches


# predections function
def pred(input_ids, attention_mask, labels):
  # Move them to the device
  input_ids = input_ids.to(device)
  attention_mask = attention_mask.to(device)
  labels = labels.to(device)

  # Forward pass with no gradient calculation
  with torch.no_grad():
    preds = model(input_ids, attention_mask).squeeze()
  return preds

In [ ]:
# Create small size tensors
patch_size = 100

input_ids_patches = split_ten(input_ids , patch_size)
attention_mask_patches = split_ten(attention_mask , patch_size)
labels_patches = split_ten(labels , patch_size)

In [ ]:
# Get and store model predections
i = 0
predections = []
for input_ids, attention_mask, labels in zip(input_ids_patches, attention_mask_patches, labels_patches):
  predection = pred(input_ids, attention_mask, labels)
  predections.extend(predection)
  i += 1


In [ ]:
# PredePredictions
len(predections)

16298

In [ ]:
f_pred = []
for i in predections:
  f_pred.append(i.tolist())

In [ ]:
# add predections to a new df
df_2 = pd.DataFrame(text_df)
df_2["price_change_predections"] = f_pred


In [ ]:
df_2.head()

,transcript,price_change_predections
0,"Prepared Remarks:\nOperator\nGood day, and wel...","[0.9999105930328369, 5.7936529628932476e-05, 3..."
1,Prepared Remarks:\nOperator\nThank you for sta...,"[1.0648688657965977e-05, 1.1954397450608667e-0..."
2,Prepared Remarks:\nOperator\nGood day and welc...,"[0.9998946189880371, 5.4282198107102886e-05, 5..."
3,"Prepared Remarks:\nOperator\nGood day, everyon...","[0.02538297139108181, 0.44049715995788574, 0.5..."
4,Prepared Remarks:\nOperator\nGreetings and wel...,"[7.830883077986073e-06, 8.876006177160889e-06,..."


In [ ]:
# save updated df
df_2.to_csv('/content/drive/MyDrive/Colab Notebooks/Scraped Motley Fool Earnings Call Transcripts/df_1.csv', index=False)

In [ ]:
# Read the data
df_2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Scraped Motley Fool Earnings Call Transcripts/df_1.csv')

In [ ]:
df_2.head()

,transcript,price_change_predections
0,"Prepared Remarks:\nOperator\nGood day, and wel...","[0.9999105930328369, 5.7936529628932476e-05, 3..."
1,Prepared Remarks:\nOperator\nThank you for sta...,"[1.0648688657965977e-05, 1.1954397450608667e-0..."
2,Prepared Remarks:\nOperator\nGood day and welc...,"[0.9998946189880371, 5.4282198107102886e-05, 5..."
3,"Prepared Remarks:\nOperator\nGood day, everyon...","[0.02538297139108181, 0.44049715995788574, 0.5..."
4,Prepared Remarks:\nOperator\nGreetings and wel...,"[7.830883077986073e-06, 8.876006177160889e-06,..."


In [ ]:
# Function to get the predections labels
def get_labels(row):
  # Convert string back to list of numbers
  numbers_list = list(map(float , row.price_change_predections[1:-1].split(",")))
  # get the index of the gratest number
  return numbers_list.index(max(numbers_list))

In [ ]:
# Apply the predections labels function
df_2["price_change_predections"] = df_2.apply(get_labels, axis=1)
df_2.head()

,transcript,price_change_predections
0,"Prepared Remarks:\nOperator\nGood day, and wel...",0
1,Prepared Remarks:\nOperator\nThank you for sta...,2
2,Prepared Remarks:\nOperator\nGood day and welc...,0
3,"Prepared Remarks:\nOperator\nGood day, everyon...",2
4,Prepared Remarks:\nOperator\nGreetings and wel...,2


# 3- NER BERT

In [ ]:
# Load the NER pipeline that was fine-tuned on earnings call data using BERT
ner_pipeline = pipeline("ner", model= AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER"),
                        tokenizer=AutoTokenizer.from_pretrained("dslim/bert-base-NER"), device=0)

def NER_financial_Filter(preprocessed_data):
    # Apply the NER pipeline to the preprocessed data to extract sentences that contain financial entities
    ner_data = ner_pipeline(preprocessed_data)
    doc = ner_pipeline(preprocessed_data)
    financial_entities = ['I-ORG', 'B-ORG', 'CURR' ]
    sentences = []
    for ent in doc:
        if ent['entity'] in financial_entities:
            start = ent['start']
            end = ent['end']
            sentence_start = preprocessed_data.rfind('.', 0, start) + 1
            sentence_end = preprocessed_data.find('.', end)
            sentence = preprocessed_data[sentence_start:sentence_end+1].strip()
            sentences.append(sentence)
    sentences = list(set(sentences))
    return str(sentences)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# 4- Sentiment analysis with BERT

In [ ]:
# Load sentiment analysis pipeline  that was fine-tuned on earnings call data using BERT
tokenizer = BertTokenizer.from_pretrained('ProsusAI/finbert')
model = BertForSequenceClassification.from_pretrained('ProsusAI/finbert')
model.to(device)

# Dealing with the large text
def chunk_text_to_window_size_and_predict_proba(txt):
    # tokenizer = BertTokenizer.from_pretrained('ProsusAI/finbert')
    # model = BertForSequenceClassification.from_pretrained('ProsusAI/finbert')
    # model.to(device)

    tokens = tokenizer.encode_plus(txt, add_special_tokens=False)
    total_len = len(tokens['input_ids'])
    input_ids = tokens['input_ids']
    attention_mask = tokens['attention_mask']

    # splits the given input text into chunks of a specified window length
    proba_list = []
    start = 0
    window_length = 510
    loop = True
    while loop:
        end = start  + window_length
        # If the end index exceeds total length, set the flag to False and adjust the end index
        if end >= total_len or total_len <= 510:
            loop = False
            end = total_len

        #1 Define the text chunk
        input_ids_chunk = input_ids[start : end]
        attention_mask_chunk = attention_mask[start : end]

        #2 Append [CLS] and [SEP]
        input_ids_chunk = [101] + input_ids_chunk + [102]
        attention_mask_chunk = [1] + attention_mask_chunk + [1]

        #3 Convert regular python list to Pytorch Tensor
        input_dict = {
            'input_ids' : torch.Tensor([input_ids_chunk]).long().to(device),
            'attention_mask' : torch.Tensor([attention_mask_chunk]).int().to(device)
        }

        #4 applies transformer model to each chunk
        outputs = model(**input_dict)

        #5 computes probabilities of each class for each chunk
        probabilities = torch.nn.functional.softmax(outputs[0], dim = -1)
        proba_list.append(probabilities)

        # print(f"start = {start}")
        # print(f"end = {end}")
        start = end
    return proba_list


# computes the mean probabilities of class predictions over all the chunks
def get_mean_from_proba(proba_list):
        #1 Reshape
        stacks = torch.stack(proba_list)
        shape = stacks.shape
        torch.reshape(stacks, (shape[0], shape[2] ) )

        #2 Ensures that gradients are not computed, saving memory
        with torch.no_grad():
            # Stack the list of tensors into a single tensor
            stacks = torch.stack(proba_list)

            # Resize the tensor to match the dimensions needed for mean computation
            stacks = stacks.resize(stacks.shape[0], stacks.shape[2])

            # Compute the mean along the zeroth dimension (i.e., the chunk dimension)
            mean = stacks.mean(dim = 0)
        return mean


# 5- Conclusion

In [ ]:
# Add dummy value
df_2['NER_Predections'] = 4
df_2['sentiment_analysis_Predections'] = 4

In [ ]:
# Applying the NER pipeline and the sentiment analysis pipeline

n = len(df_2)
print(n)
for i in range(n):
  Filtered_df = NER_financial_Filter(df_2.transcript[i])
  proba_list = chunk_text_to_window_size_and_predict_proba(Filtered_df)
  mean = get_mean_from_proba(proba_list)
  predection = torch.argmax(mean).item()
  df_2.NER_Predections[i] = Filtered_df
  df_2.sentiment_analysis_Predections[i] = predection


In [ ]:
# save updated df
df_2.to_csv('/content/drive/MyDrive/Colab Notebooks/Scraped Motley Fool Earnings Call Transcripts/df_2.csv', index=False)

    sentiment analysis Predections:
    "positive": 0,
    "negative": 1,
    "neutral": 2

In [ ]:
df_2.head()

,transcript,price_change_predections,NER_Predections,sentiment_analysis_Predections
0,"Prepared Remarks:\nOperator\nGood day, and wel...",0,"[""Some of these risks are beyond the Company's...",2
1,Prepared Remarks:\nOperator\nThank you for sta...,2,['These results are a reflection of the strate...,0
2,Prepared Remarks:\nOperator\nGood day and welc...,0,['Prepared Remarks:\nOperator\nGood day and we...,0
3,"Prepared Remarks:\nOperator\nGood day, everyon...",2,"[""Please refer to the Company's recent earning...",2
4,Prepared Remarks:\nOperator\nGreetings and wel...,2,"['In addition, on this call, the company will ...",2


In [ ]:
# Define function to make investment recommendation based on outputs
def make_recommendation(row):
    # Construct the row
    full_ticker = f"{row['sentiment_analysis_Predections']}-{row['price_change_predections']}"
    sentiment = row['sentiment_analysis_Predections']
    price = row['price_change_predections']

    # Define some example rules for making recommendation
    # You can modify or add more rules as you wish

    # Rule 1: If sentiment is positive and revenue is increasing, recommend buy
    if sentiment == 0 and price  == 0 :
        recommendation = "buy"

    # Rule 2: If sentiment is negative and revenue is decreasing, recommend sell
    elif sentiment == 1 and price == 1 :
        recommendation = "sell"

    # Rule 3: If sentiment is neutral or mixed, recommend hold
    else:
        recommendation = "hold"

    # Return recommendation
    return recommendation

In [ ]:
df_2['recommendation'] = df_2.apply(make_recommendation, axis=1)

In [ ]:
df_2.head()

,transcript,price_change_predections,NER_Predections,sentiment_analysis_Predections,recommendation
0,"Prepared Remarks:\nOperator\nGood day, and wel...",0,"[""Some of these risks are beyond the Company's...",2,hold
1,Prepared Remarks:\nOperator\nThank you for sta...,2,['These results are a reflection of the strate...,0,hold
2,Prepared Remarks:\nOperator\nGood day and welc...,0,['Prepared Remarks:\nOperator\nGood day and we...,0,buy
3,"Prepared Remarks:\nOperator\nGood day, everyon...",2,"[""Please refer to the Company's recent earning...",2,hold
4,Prepared Remarks:\nOperator\nGreetings and wel...,2,"['In addition, on this call, the company will ...",2,hold


In [ ]:
# save output
df_2.to_csv('/content/drive/MyDrive/Colab Notebooks/Scraped Motley Fool Earnings Call Transcripts/df_output.csv', index=False)